**We are going to study the dataset consisting of the data whether people were given a remainder for savings or not.**

First of all we will load the dataset and plot the basic visualization.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
dataset = files.upload()

In [ ]:
import pandas as pd
import plotly.express as px
import statistics

In [ ]:
df = pd.read_csv('savings_data_final.csv')
print (df.head())

   quant_saved  female  highschool_completed  rem_any  wealthy   age
0      13.0908       1                     0        1        0  28.0
1      39.2724       0                     1        1        1   0.0
2     294.5430       0                     1        1        1   0.0
3      58.9086       1                     1        1        1   0.0
4      78.5448       1                     1        1        1   0.0


In [ ]:
fig = px.scatter(df, y = 'quant_saved', color = 'rem_any')
fig.show()

Following observations can be made from the above graph:

*   Yellow dot represents people who were sent remainders.
*   Blue dot represents people who were not sent any remainder to save money.
*   Most of the outliers(quant_saved > 2k) are actually the people who were not sent any remainders.

Now we will take a lookat the number of peoplewho were sent remainder vs who were not sent remainder.






In [ ]:
remSent = 0
remNotSent = 0

import csv

with open ('savings_data_final.csv', newline='') as f:
  reader = csv.reader(f)
  savingsData = list(reader)

savingsData.pop(0)

for data in savingsData:
  if int(data[3]) == 0:
    remNotSent = remNotSent + 1
  else:
    remSent = remSent + 1

In [ ]:
import plotly.graph_objects as go

fig01 = go.Figure(go.Bar(x = ['Reminded', 'Not Reminded'], y = [remSent, remNotSent]))
fig01.show()

The above graph depicts that:


*   8,293 people out of 13,560 people were reminded of saving money.
*   5,267 people out of 13,560 people were not reminded of saving money.

Let's take a look at the central tendencies of the dataset.

In [ ]:
allSavings = []
for data in savingsData:
  allSavings.append(float(data[0]))

mean = statistics.mean(allSavings)
mode = statistics.mode(allSavings)
median = statistics.median(allSavings)
stdev = statistics.stdev(allSavings)

print('Mean = ' + str(mean) + '\nMedian = ' + str(median) + '\nMode = ' + str(mode)+ '\nStandard Deviation = ' + str(stdev))

Mean = 85.32780331328739
Median = 39.2724
Mode = 0.0
Standard Deviation = 196.75453011909315


In the above output, we see:


*   A huge difference between Mean and Meadian due to the outliers shifting the mean high.
*   0 as Mode, which states that most of the people have not saved any money.

Let us check if sending a remainder had any impact on the Mean, Median and Mode for the savings.


In [ ]:
reminded_savings = []
notReminded_savings = []

for data in savingsData:
  if int(data[3]) == 0:
    notReminded_savings.append(float(data[0]))
  else:
    reminded_savings.append(float(data[0]))

In [ ]:
meanOfRemindedSavings = statistics.mean(reminded_savings)
medianOfRemindedSavings = statistics.median(reminded_savings)
modeOfRemindedSavings = statistics.median(reminded_savings)
stdevOfRemindedSavings = statistics.stdev(reminded_savings)

print(meanOfRemindedSavings, medianOfRemindedSavings, modeOfRemindedSavings, stdevOfRemindedSavings)

82.56940908263547 39.2724 39.2724 173.24866414440817


In [ ]:
meanOfNotRemindedSavings = statistics.mean(notReminded_savings)
medianOfNotRemindedSavings = statistics.median(notReminded_savings)
modeOfNotRemindedSavings = statistics.median(notReminded_savings)
stdevOfNotRemindedSavings = statistics.stdev(notReminded_savings)

print(meanOfNotRemindedSavings, medianOfNotRemindedSavings, modeOfNotRemindedSavings, stdevOfNotRemindedSavings)

89.67095185226523 36.0475 36.0475 228.875050299707


As can be seen above, Mean, Median and Mode for the two categories:


*   Remainder Sent
*   Remainder Not Sent

are quite similar.
However, for standard deviation, there is a huge gap implying that the people who were not sent any remainder have saved in a very large range.


In [ ]:
import plotly.figure_factory as ff
figure = ff.create_distplot([allSavings], ['Total Savings'])
figure.show()

In [ ]:
q1 = df['quant_saved'].quantile(0.25)
q3 = df['quant_saved'].quantile(0.75)
iqr = q3 - q1
print(q1, q3, iqr)

NameError: name 'df' is not defined

In [ ]:
upperRange = q3 + 1.5 * iqr
loweRange = q1 - 1.5 * iqr

newdf = df[df['quant_saved'] < upperRange]

newFigure = ff.create_distplot([newdf['quant_saved'].tolist()], ['New Savings'], show_hist = False)
newFigure.show()

print(newdf.head())

   quant_saved  female  highschool_completed  rem_any  wealthy   age
0      13.0908       1                     0        1        0  28.0
1      39.2724       0                     1        1        1   0.0
3      58.9086       1                     1        1        1   0.0
4      78.5448       1                     1        1        1   0.0
5      39.2724       1                     1        1        1  43.0


Even after removing outliers, the dataset is still not following normal distribution, we will do data sampling.

In [ ]:
import random
newAllSavings = newdf['quant_saved'].tolist()
samples = []
for i in range(1000):
  tempList = []
  for i in range(100):
    tempList.append(random.choice(newAllSavings))

  samples.append(statistics.mean(tempList))

meanOfSamples = statistics.mean(samples)
stdevOfSamples = statistics.stdev(samples)

print('Mean= '+ str(meanOfSamples) + '\nStandard Deviation= ' + str(stdevOfSamples) )

newNewFigure = ff.create_distplot([samples], ['Sampling Distribution'], show_hist = False)
newNewFigure.show()

Mean= 46.28036298218097
Standard Deviation= 5.156350758120328


After creating Sampling Distribution, we conclude tham Standard Deviation decreased a lot and mean became approximately half.

We will perform ZScore test to see whether sending any remainder actually have any effect on the savings.

In [ ]:
notRemindedFigure = ff.create_distplot([notReminded_savings], ['Not Remainded Savings'], show_hist = False)
notRemindedFigure.show()

In [ ]:
newReminded_savings = newdf.loc[newdf['rem_any'] == 1]
newNotReminded_savings = newdf.loc[newdf['rem_any']==0]

newNotReminded_savingsList = newNotReminded_savings['quant_saved'].tolist()

In [ ]:
samples = []
for i in range(1000):
  tempList = []
  for i in range(100):
    tempList.append(random.choice(newNotReminded_savingsList))

  samples.append(statistics.mean(tempList))

meanOfNotRemindedSamples = statistics.mean(samples)
stdevOfNotRemindedSamples = statistics.stdev(samples)

print('Mean= '+ str(meanOfNotRemindedSamples) + '\nStandard Deviation= ' + str(stdevOfNotRemindedSamples) )

notRemindedSamplesFigure = ff.create_distplot([samples], ['Sampling Distribution'], show_hist = False)
notRemindedSamplesFigure.show()

Mean= 43.95082396454505
Standard Deviation= 4.931112289394365


ZScore = (meanOfSamplingReminded - meanOfSamplingNotReminded)/stdevOfSamplingNotReminded)